In [1]:
#import packages

import matplotlib, plotly, pandas, numpy, dash, datetime, PyQt6, sys, os
from PyQt6.QtWidgets import QApplication, QLabel, QSplashScreen
from PyQt6.QtCore import Qt, QTimer
from PyQt6.QtGui import QPixmap
from PyQt6.QtWidgets import QApplication, QWidget, QLabel
from PyQt6.QtGui import QIcon, QFont, QPixmap, QMovie, QRegion
from PyQt6.QtCore import Qt
import sys
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QProgressBar, QVBoxLayout
from PyQt6.QtGui import QPixmap
from PyQt6.QtCore import Qt, QTimer
import openpyxl
import xlrd
import pandasql as ps
import sys
import pandas as pd
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout, QProgressBar, QPushButton, QFileDialog,  QProgressBar, QPushButton, QFileDialog, QMessageBox, QTextEdit
from PyQt6.QtGui import QPixmap, QFont
from PyQt6.QtCore import Qt, QTimer, pyqtSlot
from PyQt6.QtWidgets import QApplication, QTextEdit, QPushButton, QVBoxLayout, QHBoxLayout, QWidget, QLineEdit, QTableWidget, QTableWidgetItem
from PyQt6.QtGui import QTextCursor
from PyQt6.QtCore import QMimeData, Qt
from PyQt6.QtGui import QTextCursor
import resources_rc



In [2]:


username = os.environ.get('USERNAME')  # Replace this with the actual username or use os.environ.get('USERNAME')

class ProgressWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setGeometry(200, 200, 700, 250)
        self.setWindowTitle("Simba Tollator")
        self.setWindowIcon(QIcon(':/Resources/Simba.png'))
        # Create the main layout (vertical layout)
        main_layout = QVBoxLayout(self)
        main_layout.setAlignment(Qt.AlignmentFlag.AlignTop | Qt.AlignmentFlag.AlignHCenter)

        # Create and set the background image using QLabel
        label = QLabel(self)
        pixmap = QPixmap(':/Resources/Simba.png')
        label.setPixmap(pixmap)
        label.setGeometry(0, 0, 700, 200)
        label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        main_layout.addWidget(label)

        # Create a QLabel for the welcome message
        welcome_label = QLabel(f"{username}! Welcome to Simba Toll-a-Tor", self)
        welcome_label.setGeometry(100, 250, 500, 50)
        welcome_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        welcome_font = QFont("Arial", 18, QFont.Weight.Bold)  # Set the font size and weight
        welcome_label.setFont(welcome_font)

        # Create the progress bar
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setGeometry(100, 300, 500, 20)
        main_layout.addWidget(self.progress_bar)

        # Use QTimer to update the progress bar every 100 milliseconds
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_progress_bar)
        self.timer.start(70)  # 100 milliseconds (0.1 second)

        self.progress_value = 0

    def update_progress_bar(self):
        # Update the progress bar value
        self.progress_value += 1
        self.progress_bar.setValue(self.progress_value)

        # When the progress bar reaches 100, stop the timer and show the file browsing GUI
        if self.progress_value == 100:
            self.timer.stop()
            self.show_file_browsing()

    @pyqtSlot()
    def show_file_browsing(self):
        self.file_window = FileBrowsingWindow()
        self.file_window.show()
        self.close()

class LicenseWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setGeometry(200, 200, 400, 150)
        self.setWindowTitle("Simba Tollator - License Key")
        self.setWindowIcon(QIcon(':/Resources/Simba.png'))
        layout = QVBoxLayout(self)

        # License Input
        self.license_label = QLabel("Enter your license key:")
        layout.addWidget(self.license_label)

        self.license_input = QLineEdit(self)
        layout.addWidget(self.license_input)

        # Validate Button
        self.validate_button = QPushButton("Validate", self)
        self.validate_button.clicked.connect(self.validate_license)
        layout.addWidget(self.validate_button)

    def validate_license(self):
        key = self.license_input.text()
        valid_key = "JUDX-5V31-HGYA-R1LC"  # This is a hardcoded sample. Ideally, you'd want a dynamic and secure method.

        if key == valid_key:
            self.progress_window = ProgressWindow()
            self.progress_window.show()
            self.close()
        else:
            QMessageBox.warning(self, "Invalid Key", "The provided license key is invalid. Please try again.")

class ResultSummaryWindow(QWidget):
    def __init__(self, dataframe):
        super().__init__()

        self.df = dataframe
        self.setGeometry(200, 200, 500, 400)
        self.setWindowTitle("Result Summary")
        self.setWindowIcon(QIcon(':/Resources/Simba.png'))

        # Create a QTableWidget to display the summary
        self.table = QTableWidget(self)

        self.layout = QVBoxLayout(self)
        self.layout.addWidget(self.table)

        # Initialize the grand_total_label
        self.grand_total_label = QLabel(self)
        self.layout.addWidget(self.grand_total_label)

        # Declare admin_fee_total_label without text here
        self.admin_fee_total_label = QLabel(self)
        self.layout.addWidget(self.admin_fee_total_label)
        
        # Move the search box and button beneath the grand totals
        self.search_input = QLineEdit(self)
        self.search_btn = QPushButton("Search", self)
        self.search_btn.clicked.connect(self.search_summary)
        self.layout.addWidget(self.search_input)
        self.layout.addWidget(self.search_btn)

        # Setup layout
        self.setLayout(self.layout)

        # Populate the table (make sure this comes after defining grand_total_label and admin_fee_total_label)
        self.populate_table()

    def populate_table(self):
        # Group by "Res." and calculate the number of rows and the sum of toll cost
        summary = self.df.groupby('Res.').agg(Num_of_Rows=('Res.', 'size'), Sum_of_Toll_Cost=('Trip Cost', 'sum')).reset_index()
        grand_total = summary["Sum_of_Toll_Cost"].sum()
        summary['Admin Fee'] = summary['Num_of_Rows'] * 5
        admin_fee_total=summary['Admin Fee'].sum()

        self.table.setColumnCount(4)
        self.table.setRowCount(len(summary))
        self.table.setHorizontalHeaderLabels(['Res.', 'Number of tolls', 'Sum Admin Fee','Sum of Toll Cost'])

        for i, (idx, row) in enumerate(summary.iterrows()):
            self.table.setItem(i, 0, QTableWidgetItem(str(row['Res.'])))
            self.table.setItem(i, 1, QTableWidgetItem(str(row['Num_of_Rows'])))
            self.table.setItem(i, 2, QTableWidgetItem(f"${row['Admin Fee']:.2f}"))
            self.table.setItem(i, 3, QTableWidgetItem(f"${row['Sum_of_Toll_Cost']:.2f}"))
         # Grand total labels (both the admin fee and the actual grand total)
        grand_total_style = "font-size: 16px; font-weight: bold;"
                    
        self.grand_total_label.setStyleSheet(grand_total_style)
        self.grand_total_label.setText(f"Grand Toll Total: ${grand_total:.2f}")
                    
        self.admin_fee_total_label.setStyleSheet(grand_total_style)
        self.admin_fee_total_label.setText(f"Admin Fee Total: ${admin_fee_total:.2f}")
    def search_summary(self):
            query = self.search_input.text().strip()
            if query:
                # Check if the Res. number exists in our dataframe
                if query in self.df['Res.'].tolist():
                    # Open the RecordBrowser and display the record for the given Res. number
                    self.record_browser = RecordBrowser(self.df, [query])
                    self.record_browser.show()
                else:
                    QMessageBox.warning(self, "Not Found", f"No record found for Res. number: {query}")
            else:
                QMessageBox.warning(self, "Search Error", "Please enter a Res. number to search.")

    def show(self):
        super().show()


            
class FileBrowsingWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setGeometry(200, 200, 700, 400)
        self.setWindowTitle("Simba Tollator - File Browsing")
        self.setWindowIcon(QIcon(':/Resources/Simba.png'))
        
        # Create the main layout (vertical layout)
        main_layout = QVBoxLayout(self)
        main_layout.setAlignment(Qt.AlignmentFlag.AlignTop | Qt.AlignmentFlag.AlignHCenter)

        # Create the Browse Buttons
        self.rcm_browse_button = QPushButton("Browse for RCM File", self)
        self.linkt_browse_button = QPushButton("Browse for Linkt Tolls List", self)
        self.rcm_browse_button.setFont(QFont("Arial", 12))
        self.linkt_browse_button.setFont(QFont("Arial", 12))
        self.rcm_browse_button.clicked.connect(self.browse_rcm)
        self.linkt_browse_button.clicked.connect(self.browse_linkt)
        main_layout.addWidget(self.rcm_browse_button)
        main_layout.addWidget(self.linkt_browse_button)


        self.summary_button = QPushButton("View Summary", self)
        self.summary_button.setFont(QFont("Arial", 12))
        self.summary_button.clicked.connect(self.show_summary)
        main_layout.addWidget(self.summary_button)
        self.summary_button.setEnabled(False) # Initially disabled until both files are selected

        # ...

        # Add to the end of check_files_loaded method:
        self.summary_button.setEnabled(True)

        # Add the Continue button
        self.continue_button = QPushButton("Continue", self)
        self.continue_button.setFont(QFont("Arial", 12))
        self.continue_button.clicked.connect(self.show_records)
        main_layout.addWidget(self.continue_button)

        self.continue_button.setEnabled(False) # Initially disabled until both files are selected

            # Create QLabel for displaying RCM DataFrame top 3 rows
        self.rcm_data_label = QLabel("RCM DataFrame (Top 3 rows):", self)
        self.rcm_data_label.setFont(QFont("Arial", 12))
        main_layout.addWidget(self.rcm_data_label)

        self.rcm_data_text = QTextEdit(self)
        self.rcm_data_text.setReadOnly(True)
        main_layout.addWidget(self.rcm_data_text)

        # Create QLabel for displaying Linkt DataFrame top 3 rows
        self.linkt_data_label = QLabel("Linkt DataFrame (Top 3 rows):", self)
        self.linkt_data_label.setFont(QFont("Arial", 12))
        main_layout.addWidget(self.linkt_data_label)

        self.linkt_data_text = QTextEdit(self)
        self.linkt_data_text.setReadOnly(True)
        main_layout.addWidget(self.linkt_data_text)

    def show_summary(self):
        self.summary_window = ResultSummaryWindow(result_df)
        self.summary_window.show()


    @pyqtSlot()
    def browse_rcm(self):
        # Ask the user to select the RCM file
        QMessageBox.information(self, "Select RCM File", "Please select the RCM file.")
        # Open file dialog to select RCM file
        rcm_file, _ = QFileDialog.getOpenFileName(self, "Select RCM File", "", "RCM Files (*.xlsx)")
        if rcm_file:
            print("RCM File selected:", rcm_file)
            global rcm_df
            rcm_df = pd.read_excel(rcm_file,header=None)
            rcm_df.columns=rcm_df.iloc[2]
            rcm_df=rcm_df.iloc[3:]
            rcm_df.reset_index(drop=True, inplace=True)
            # Extracting the part after the space
            rcm_df['Vehicle'] = rcm_df['Vehicle'].str.split(' ', n=1).str.get(1)

            # Removing everything after a decimal point, if present
            rcm_df['Vehicle'] = rcm_df['Vehicle'].str.split('.').str.get(0)
            # Removing leading zeros
            rcm_df['Vehicle'] = rcm_df['Vehicle'].str.lstrip('0')
            # Now convert the values to integers if possible, else keep the original value
            try:
                rcm_df['Vehicle'] = rcm_df['Vehicle'].astype(int)
            except:
                pass
            rcm_df['Pickup Date Time'] = pd.to_datetime(rcm_df['Pickup Date'] + ' ' + rcm_df['Time_c13']).dt.strftime('%Y-%m-%d %H:%M:%S')
            rcm_df['Pickup Date Time'] = pd.to_datetime(rcm_df['Pickup Date Time']).dt.strftime('%Y-%m-%d %H:%M:%S')
            rcm_df['Dropoff Date Time'] = pd.to_datetime(rcm_df['Dropoff Date'] + ' ' + rcm_df['Time']).dt.strftime('%Y-%m-%d %H:%M:%S')
            rcm_df['Dropoff Date Time'] = pd.to_datetime(rcm_df['Dropoff Date Time']).dt.strftime('%Y-%m-%d %H:%M:%S')
            rcm_df=rcm_df.drop_duplicates()
            print("RCM DataFrame:")
            print(rcm_df)
                    # Format RCM DataFrame top 3 rows as a table
            rcm_table_html = rcm_df.head(3).to_html(index=False, border=1)
            self.rcm_data_text.setHtml(f"<table>{rcm_table_html}</table>")
                # If both files are loaded, enable Continue button
        self.check_files_loaded()


    @pyqtSlot()
    def browse_linkt(self):
        # Ask the user to select the Linkt tolls list file
        global linkt_df, result_df, res_numbers
      
        QMessageBox.information(self, "Select Linkt Tolls List", "Please select the Linkt tolls list file.")
        # Open file dialog to select Linkt tolls list file
        linkt_file, _ = QFileDialog.getOpenFileName(self, "Select Linkt Tolls List", "", "Linkt Files (*.xls)")
        if linkt_file:
            print("Linkt Tolls List File selected:", linkt_file)
            linkt_df = pd.read_excel(linkt_file)
            linkt_df['Start Date'] = pd.to_datetime(linkt_df['Start Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
            linkt_df['End Date'] = pd.to_datetime(linkt_df['End Date']).dt.strftime('%Y-%m-%d %H:%M:%S')
            linkt_df['Trip Cost']=linkt_df['Trip Cost'].astype(str)
            linkt_df['Trip Cost']=linkt_df['Trip Cost'].str.replace(r'[^0-9.]', '', regex=True)
            try:
                linkt_df['LPN/Tag number'] = linkt_df['LPN/Tag number'].astype(int)
            except:
                pass
            print("Linkt DataFrame:")
            print(linkt_df)
            linkt_df=linkt_df.drop_duplicates()
            linkt_df['Trip Cost']=linkt_df['Trip Cost'].astype(float, errors='ignore')
            linkt_table_html = linkt_df.head(3).to_html(index=False, border=1)
            self.linkt_data_text.setHtml(f"<table>{linkt_table_html}</table>")
            
        self.check_files_loaded()


    def check_files_loaded(self):
        if 'rcm_df' in globals() and 'linkt_df' in globals():
            query = """
                SELECT *
                FROM linkt_df
                JOIN rcm_df 
                ON linkt_df.[LPN/Tag number] = rcm_df.[Vehicle]
                WHERE linkt_df.[Start Date] BETWEEN rcm_df.[Pickup Date Time] AND rcm_df.[Dropoff Date Time]
            """
            global result_df, res_numbers
            result_df = ps.sqldf(query, globals())
            result_df=result_df.drop_duplicates()
            res_numbers = result_df['Res.'].unique().tolist()
            self.continue_button.setEnabled(True)

    @pyqtSlot()
    def show_records(self):
        # Open the RecordBrowser with result_df and res_numbers
        self.record_browser = RecordBrowser(result_df, res_numbers)
        self.record_browser.show()
        self.close()



class RecordBrowser(QWidget):
    def __init__(self, dataframe, res_numbers):
        super().__init__()

        self.df = dataframe
        self.res_numbers = res_numbers
        self.current_index = 0

        self.setGeometry(200, 200, 800, 600)
        self.setWindowTitle("Record Browser")
        self.setWindowIcon(QIcon(':/Resources/Simba.png'))
        self.main_layout = QVBoxLayout(self)

       
        # Add text area for displaying the record
        self.record_display = QTextEdit(self)
        self.record_display.setReadOnly(True)
        self.main_layout.addWidget(self.record_display)

        # Add navigation buttons
        self.nav_layout = QHBoxLayout()

        self.prev_button = QPushButton("Previous", self)
        self.next_button = QPushButton("Next", self)
        self.prev_button.clicked.connect(self.show_prev)
        self.next_button.clicked.connect(self.show_next)

        self.nav_layout.addWidget(self.prev_button)
        self.nav_layout.addWidget(self.next_button)
        self.main_layout.addLayout(self.nav_layout)
        # Create an input field for entering 'Res.' number
        self.res_input = QLineEdit(self)
        self.res_input.setPlaceholderText("Enter Res. number...")
        self.main_layout.addWidget(self.res_input)
                # Connect the returnPressed signal for the QLineEdit
        self.res_input.returnPressed.connect(self.search_res)


        # Create a 'Search' button and connect it to a method
        self.search_button = QPushButton("Search", self)
        self.search_button.clicked.connect(self.search_res)
        self.main_layout.addWidget(self.search_button)


        # Add Copy button
        self.copy_button = QPushButton("Copy Table", self)
        self.copy_button.clicked.connect(self.copy_table_to_clipboard)
        self.main_layout.addWidget(self.copy_button)
        
        self.view_summary_btn = QPushButton("View Summary", self)
        self.view_summary_btn.clicked.connect(self.open_summary_window)
        self.main_layout.addWidget(self.view_summary_btn)

        # Display the first record
        self.show_record()

    def show_record(self):
        # Clear the existing content
        self.record_display.clear()
        self.record_display.setStyleSheet("background-color: black; color: white;")

        if self.res_numbers:
            current_res_number = self.res_numbers[self.current_index]
            
            # Display the current Res. as a header
            self.record_display.insertHtml(f"<h2 style='color:white;'>Contract (Res.): {current_res_number}</h2><hr style='border-color:white;'>")
            
            records = self.df[self.df['Res.'] == current_res_number]

            # Display the total number of rows (tolls used) and the sum of Trip Cost
            num_rows = len(records)
            total_cost = records['Trip Cost'].sum()
            toll_usage_cost=num_rows*5
            total_toll_cost=total_cost+toll_usage_cost
            
            # Using inline CSS to adjust for white font color.
            self.record_display.append(f"<span style='color:white; font-size:16px;'>Tolls used: {num_rows}</span>")
            self.record_display.append(f"<span style='color:white; font-size:16px;'>Total Trip Cost: ${total_cost:.2f}</span>")
            self.record_display.append(f"<span style='color:white; font-size:16px;'>Total Toll Cost: ${toll_usage_cost:.0f}</span>")
            self.record_display.append(f"<span style='color:white; font-size:16px;'>Total  Cost: ${total_toll_cost:.2f}</span>")

            # Display the records as an HTML table. Ensure all elements in table are also styled white.
            table_html = records.to_html(index=False, border=1, classes='table-dark')
            table_html = table_html.replace('<table', '<table style="color:white;"')
            self.record_display.insertHtml(table_html)
        else:
            self.record_display.setText("No records available")

    def search_res(self):
    # Get the Res. number entered by the user
        res_num = self.res_input.text()

        if not res_num:
            QMessageBox.warning(self, "Search Error", "Please enter a Res. number to search.")
            return

        # Check if the Res. number exists in our list of numbers
        if res_num in self.res_numbers:
            self.current_index = self.res_numbers.index(res_num)
            self.show_record()
        else:
            QMessageBox.warning(self, "Not Found", f"No record found for Res. number: {res_num}")

    def open_summary_window(self):
        self.summary_window = ResultSummaryWindow(self.df)
        self.summary_window.show()

    def copy_table_to_clipboard(self):
        if self.res_numbers:
            current_res_number = self.res_numbers[self.current_index]
            records = self.df[self.df['Res.'] == current_res_number]
            clipboard_content = "Start date -------------- Details ------------- Trip Cost\n"
            for _, row in records.iterrows():
                line = f"{row['Start Date']} --------------{row['Details']}--------------${row['Trip Cost']}\n"
                clipboard_content += line
            clipboard = QApplication.clipboard()
            clipboard.setText(clipboard_content)

    def show_prev(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_record()

    def show_next(self):
        if self.current_index < len(self.res_numbers) - 1:
            self.current_index += 1
            self.show_record()

    
if __name__ == "__main__":
    app = QApplication(sys.argv)
    # Show the LicenseWindow first
    license_window = LicenseWindow()
    license_window.show()
    sys.exit(app.exec())


RCM File selected: C:/Users/bkamalal/OneDrive - onlineeportal/Desktop/Simba Intro/Daily Activity to 19 Aug 2023 1903.xlsx
RCM DataFrame:
2        #  Res.      Ref.  Update Notes           Status  \
0        1  4538  64980557     NaN  View  Returned          
1        2  4614       NaN  Update  View  Returned          
2        3  4143       NaN  Update  View  Returned          
3        4  3113       NaN  Update  View  Returned          
4        5  4375       NaN  Update  View  Returned          
...    ...   ...       ...     ...   ...              ...   
1218  1219  5091       NaN  Update  View  Reservation       
1219  1220  5701       NaN  Update  View  Reservation       
1220  1221  7692       NaN  Update  View  Reservation       
1221  1222  7652       NaN  Update  View  Reservation       
1222   NaN   NaN       NaN     NaN   NaN              NaN   

2                                            Dropoff  Day Dropoff Date   Time  \
0      SYDOffice: 935 Botany Road, Rosebery 2018 

SystemExit: 0

c:\Users\bkamalal\Simba\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<class 'pandas.core.frame.DataFrame'>
Index: 4028 entries, 0 to 4337
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Start Date      4028 non-null   object        
 1   Details         4028 non-null   object        
 2   LPN/Tag number  4028 non-null   object        
 3   Vehicle Class   4028 non-null   int64         
 4   Trip Cost       4028 non-null   object        
 5   Fleet ID        0 non-null      float64       
 6   End Date        4028 non-null   object        
 7   Date            4028 non-null   datetime64[ns]
 8   Rego            4028 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 314.7+ KB


In [3]:
import pandas as pd 

tolls_df=pd.read_excel("/Users/Bardia/Desktop/My Toll Test Files/6 May/TOLLS till 06 May 24.xls")

In [5]:
tolls_df['End Date'] = pd.to_datetime(tolls_df['End Date'], format="%d %b %Y %I:%M%p")
tolls_df['End Date'] = tolls_df['End Date'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [6]:
tolls_df.to_csv("checking1.csv")